Load News Data

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

df = pd.read_csv('../data/raw_analyst_ratings.csv')

#droping nameless index column
df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')

def remove_timezone(x):
    if isinstance(x, pd.Timestamp) and x.tzinfo is not None:
        return x.tz_convert(None)
    elif isinstance(x, datetime) and x.tzinfo is not None:
        return x.replace(tzinfo=None)
    return x

df['date'] = pd.to_datetime(df['date'], format='mixed', errors='coerce')
df.dropna(subset=['date'], inplace=True)
df['date'] = df['date'].apply(remove_timezone)

# Set date as index and sort
df.set_index('date', inplace=True)
df.sort_index(inplace=True)
print(df.columns)   # Should show headline, url, publisher, stock
print(df.index) 

df.head(1)

C:\Users\Tinsae D.D\AppData\Local\Temp\ipykernel_5556\1863515706.py:17: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['date'] = pd.to_datetime(df['date'], format='mixed', errors='coerce')


Index(['headline', 'url', 'publisher', 'stock'], dtype='object')
DatetimeIndex(['2009-02-14 00:00:00', '2009-04-27 00:00:00',
               '2009-04-27 00:00:00', '2009-04-29 00:00:00',
               '2009-05-22 00:00:00', '2009-05-27 00:00:00',
               '2009-05-27 00:00:00', '2009-05-27 00:00:00',
               '2009-05-27 00:00:00', '2009-05-27 00:00:00',
               ...
               '2020-06-11 15:32:46', '2020-06-11 16:15:38',
               '2020-06-11 16:24:41', '2020-06-11 16:25:21',
               '2020-06-11 16:30:31', '2020-06-11 16:49:41',
               '2020-06-11 16:51:33', '2020-06-11 17:01:39',
               '2020-06-11 17:11:20', '2020-06-11 17:12:35'],
              dtype='datetime64[ns]', name='date', length=1407328, freq=None)


,headline,url,publisher,stock
date,,,,
2009-02-14,How Treasuries and ETFs Work,https://www.benzinga.com/28044/how-treasuries-...,Paco Ahlgren,NAV
